# Tugas 1 - Crawling Data (detik.com)

Crawling data adalah proses pengambilan data yang digunakan untuk mengumpulkan informasi dari situs web. proses ini dilakukan oleh program komputer yang disebut **"crawler" atau "spider,"** yang menavigasi situs web dan mengekstrak data yang diperlukan.

Pada tugas ini menggunakan **"BeautifulSoup"** sebagai library Crawler dan proses pengambilan data diambil dari website **detik.com**

In [1]:
judul = []
tanggal = []
isi = []
url_list = []
kategori_list = []

Langkah Pertama Membuat list kosong untuk menyimpan data dan menampilkan proses crawling

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, max_articles):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= max_articles:
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        url_list.append(link)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://food.detik.com/indeks"]
categories = ["Kuliner"]

max_articles = 10

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    if len(judul) >= max_articles:
        break

    # Looping untuk beralih halaman
    for page in range(1, 4):
        if len(judul) >= max_articles:
            break

        url = f"{base_url}/{page}"
        get_data(url, category, max_articles)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "isi": isi, "tanggal": tanggal, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

Salt Bae Masak BBQ hingga Ciri Tempat Makan Enak dari Netizen
3 Resep Udang Goreng ala Restoran yang Terkenal Lezatnya
Sari Delicatessen yang Legendaris Kembali Hadir dengan Konsep Baru
Minum Air Garam Hangat Tiap Hari Punya 9 Manfaat Sehat Ini
7 Racikan Kopi Terburuk di Dunia Diracik Pakai Bahan Ini
5 Manfaat Daun Pepaya yang Rasanya Pahit Nikmat
5 Makanan Penguat Imunitas, Cocok untuk Cegah Monkeypox
Lagi Diet, Bagusnya Makan Putih atau Kuning Telur?
Manis! Ultah Al Ghazali ke-27 Dirayakan dengan Makan Malam Keluarga
Dinar Candy Traktir Penghuni Rumah Singgah dan Kulineran ke Kamboja


Data yang diambil terdiri dari :
- Judul
- Isi berita
- Tanggal
- Kategori

Pada tahap ini menampilkan data tersebut dan diambil berdasarkan kategori ***Kuliner*** dan hanya mengambil 10 sample data crawl

In [3]:
df=pd.read_csv("data_berita_detik.csv")
df

,judul,isi,tanggal,kategori
0,Salt Bae Masak BBQ hingga Ciri Tempat Makan En...,Jakarta -\n \n Aksi Salt Bae mas...,"Rabu, 04 Sep 2024 09:30 WIB",Kuliner
1,3 Resep Udang Goreng ala Restoran yang Terkena...,Tentang\nBahan\nLangkah\n\nJakarta - Udang gor...,"Rabu, 04 Sep 2024 09:00 WIB",Kuliner
2,Sari Delicatessen yang Legendaris Kembali Hadi...,Jakarta - Sari Delicatessen and Cafe salah sat...,"Rabu, 04 Sep 2024 08:30 WIB",Kuliner
3,Minum Air Garam Hangat Tiap Hari Punya 9 Manfa...,Jakarta -\n \n Memulai hari bisa...,"Rabu, 04 Sep 2024 08:00 WIB",Kuliner
4,7 Racikan Kopi Terburuk di Dunia Diracik Pakai...,Jakarta -\n \n Racikan kopi begi...,"Rabu, 04 Sep 2024 07:30 WIB",Kuliner
5,5 Manfaat Daun Pepaya yang Rasanya Pahit Nikmat,Jakarta - Daun pepaya disukai orang Indonesia ...,"Rabu, 04 Sep 2024 07:00 WIB",Kuliner
6,"5 Makanan Penguat Imunitas, Cocok untuk Cegah ...",Jakarta -\n \n Maraknya isu peny...,"Rabu, 04 Sep 2024 06:00 WIB",Kuliner
7,"Lagi Diet, Bagusnya Makan Putih atau Kuning Te...",Jakarta - Telur banyak dijadikan pilihan sumbe...,"Rabu, 04 Sep 2024 05:00 WIB",Kuliner
8,Manis! Ultah Al Ghazali ke-27 Dirayakan dengan...,Jakarta - Ulang tahun Al Ghazali yang ke-27 di...,"Selasa, 03 Sep 2024 22:00 WIB",Kuliner
9,Dinar Candy Traktir Penghuni Rumah Singgah dan...,"Jakarta - Resmi menyandang gelar sarjana, Dina...","Selasa, 03 Sep 2024 21:00 WIB",Kuliner
